In [1]:
import os
import nibabel as nib
import numpy as np
import pandas as pd
import sys

# -----------------------------
# CONFIGURACIÓN
# -----------------------------
NNUNET_DIR = "/media/guest/PORT-DISK/Practicas/MicroBleeds/nnUNet_raw_data_dou/Dataset002_Dou2016"
imagesTr_dir = os.path.join(NNUNET_DIR, "imagesTr")
labelsTr_dir = os.path.join(NNUNET_DIR, "labelsTr")

# Definición de modalidades
MODALITIES = {
    "T2*": "_0000",
    "Mask": ""  # La máscara está en otra carpeta y no tiene sufijo numérico
}

def get_nifti_metadata(path, modality_name, sub_id):
    """Extrae metadatos del header sin cargar toda la imagen (más rápido)."""
    if not os.path.exists(path):
        return None

    try:
        nii = nib.load(path)
        header = nii.header
        
        # 1. Dimensiones (X, Y, Z)
        dims = nii.shape 
        
        # 2. Tamaño de Voxel (Spacing en mm)
        zooms = header.get_zooms()
        
        # 3. Orientación (RAS, LPS, etc.)
        orientation = nib.aff2axcodes(nii.affine)
        
        # 4. Tipo de dato
        dtype = header.get_data_dtype()

        return {
            "Subject": sub_id,
            "Modality": modality_name,
            "Dim_X": dims[0],
            "Dim_Y": dims[1],
            # Manejo seguro si la imagen es 2D o tiene dimension temporal
            "Dim_Z (Slices)": dims[2] if len(dims) > 2 else 1,
            "Voxel_X (mm)": round(zooms[0], 4),
            "Voxel_Y (mm)": round(zooms[1], 4),
            # Manejo seguro de voxel Z
            "Voxel_Z (mm)": round(zooms[2], 4) if len(zooms) > 2 else 1.0,
            "Orientation": "".join(orientation),
            "DType": dtype
        }
    except Exception as e:
        print(f"Error leyendo {path}: {e}")
        return None

# -----------------------------
# EJECUCIÓN
# -----------------------------

# 0. Verificación de seguridad
if not os.path.exists(imagesTr_dir):
    print(f"ERROR CRÍTICO: No se encuentra el directorio: {imagesTr_dir}")
    sys.exit(1)

# 1. Obtener lista de sujetos únicos
files = os.listdir(imagesTr_dir)

# MEJORA: Usamos replace en lugar de split para evitar errores si el ID tiene guiones bajos (ej: sub_01)
sub_ids = sorted([f.replace("_0000.nii.gz", "") for f in files if f.endswith("_0000.nii.gz")])

print(f"Detectados {len(sub_ids)} sujetos en {imagesTr_dir}. Iniciando análisis...\n")

data_list = []

for sub_id in sub_ids:
    # Iterar por cada modalidad (T2*, Mask)
    for mod_name, suffix in MODALITIES.items():
        
        # Determinar ruta dependiendo si es imagen o label
        if mod_name == "Mask":
            filename = f"{sub_id}.nii.gz"
            path = os.path.join(labelsTr_dir, filename)
        else:
            filename = f"{sub_id}{suffix}.nii.gz"
            path = os.path.join(imagesTr_dir, filename)
            
        # Extraer datos
        meta = get_nifti_metadata(path, mod_name, sub_id)
        if meta:
            data_list.append(meta)
        else:
            # Solo advertir si falta la máscara, a veces hay datasets incompletos
            print(f"Warning: Falta {mod_name} para sujeto {sub_id}")

# 2. Crear DataFrame de Pandas
if not data_list:
    print("No se encontraron datos. Revisa las rutas.")
    sys.exit(0)

df = pd.DataFrame(data_list)
      
# Agrupamos por modalidad para ver si todos tienen la misma resolución
group_cols = ["Dim_X", "Dim_Y", "Dim_Z (Slices)", "Voxel_X (mm)", "Voxel_Y (mm)", "Voxel_Z (mm)"]

for mod in MODALITIES.keys():
    print(f"\n--- Análisis de Modalidad: {mod} ---")
    df_mod = df[df["Modality"] == mod]
    
    if df_mod.empty:
        print("  No hay datos para esta modalidad.")
        continue

    # Contamos cuántas combinaciones únicas de dimensiones/voxels existen
    unique_configs = df_mod[group_cols].value_counts()
    
    print(unique_configs)
    
    # Chequeo de Orientación
    orientations = df_mod["Orientation"].unique()
    if len(orientations) > 1:
        print(f"¡ALERTA! Orientaciones mixtas detectadas: {orientations}")
    else:
        print(f"Orientación consistente: {orientations[0]}")

# 3. Exportar a CSV
output_csv = "/media/guest/PORT-DISK/Practicas/MicroBleeds/analisis_resolucion_dataset_dou.csv"

# Asegurar que el directorio de salida existe
output_dir = os.path.dirname(output_csv)
if not os.path.exists(output_dir):
    try:
        os.makedirs(output_dir)
        print(f"Directorio creado: {output_dir}")
    except OSError:
        print(f"No se pudo crear el directorio {output_dir}. Guardando en directorio actual.")
        output_csv = "analisis_resolucion_dataset_dou.csv"

df.to_csv(output_csv, index=False)
print(f"\nAnálisis completo guardado en: {output_csv}")

# 4. Mostrar primeras filas del análisis:
print("\nPrimeras filas del análisis:")
print(df.head(10))

Detectados 20 sujetos en /media/guest/PORT-DISK/Practicas/MicroBleeds/nnUNet_raw_data_dou/Dataset002_Dou2016/imagesTr. Iniciando análisis...


--- Análisis de Modalidad: T2* ---
Dim_X  Dim_Y  Dim_Z (Slices)  Voxel_X (mm)  Voxel_Y (mm)  Voxel_Z (mm)
512    512    150             0.4492        0.4492        1.0000          10
                                                          0.9999           3
                                                          1.0001           2
342    410    150             0.4492        0.4492        1.0000           1
368    416    147             0.4492        0.4492        1.0000           1
357    407    141             0.4492        0.4492        1.0000           1
343    409    147             0.4492        0.4492        1.0000           1
374    397    147             0.4492        0.4492        1.0000           1
Name: count, dtype: int64
¡ALERTA! Orientaciones mixtas detectadas: ['LPS' 'RAS' 'LAS']

--- Análisis de Modalidad: Mask ---
Dim_X  Dim